<a href="https://colab.research.google.com/github/liminovna/palladizator/blob/colab_version/palladizator_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Добро пожаловать в "Палладизатор"! 

С помощью этой простенькой программы вы можете транскрибировать текст с китайского языка на русский.

Для этого достаточно сделать несколько простых шагов:

1. Первым делом нужно установить необходимые компоненты, нажав на кнопку ▶ в ячейке ниже. Этот шаг нужно выполнить всего один раз.

In [ ]:
!git clone -b colab_version https://github.com/liminovna/palladizator.git palladizator

%cd palladizator

!pip install -r requirements.txt

2. Нажимаем ▶, чтобы импортировать библиотеки и сделать другие важные штуки. Это также достаточно сделать всего один раз.

In [ ]:
from palladius_colab import zh_phrase

3. Готово! Теперь вы можете вставить текст, который нужно транскрибировать, и выбрать опцию cpitalize, если хотите, чтобы каждое слово начиналось с большой буквы!

Например, при включенной опции результат для фразы "国家语委现代汉语平衡语料库" будет "Гоцзя Юй Вэй Сяньдай Ханьюй Пинхэн Юйляоку", а при выключенной - "гоцзя юй вэй сяньдай ханьюй пинхэн юйляоку".

In [ ]:
phrase = '' #@param {type:"string"}
capitalize = True #@param {type:"boolean"}

s = zh_phrase(phrase, capitalize)

print(s.get_ru_text())

Спасибо, что воспользовались "Палладизатором"!

При возникновении трудностей пишите на vladaliminska @ gmail.com